In [5]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
%matplotlib inline

In [6]:
df = pd.read_csv('Resources/US_French_Cab_Bordeauxs.csv')
df

,ID,alcohol,category,country,description,designation,price,price_bin,rating,rating_bin,region,subregion,subsubregion,title,url,varietal,vintage,winery
0,1,13.2,Red,US,This is among winemaker Cathy Corison's finest...,Kronos Vineyard,185.0,$100+,95,91-100,California,Napa,St. Helena,Corison 2015 Kronos Vineyard Cabernet Sauvigno...,https://www.winemag.com/buying-guide/corison-2...,Cabernet Sauvignon,2015,Corison
1,52,14.2,Red,US,This bottling by industry veterans John and He...,NaN,35.0,$25-$50,94,91-100,California,Central Coast,Paso Robles,Falcone 2016 Cabernet Sauvignon (Paso Robles),https://www.winemag.com/buying-guide/falcone-2...,Cabernet Sauvignon,2016,Falcone
2,128,14.5,Red,US,"Inky in the glass, this rich and structured bo...",Reserve,56.0,$51-$100,94,91-100,California,Central Coast,Paso Robles,Daou 2016 Reserve Cabernet Sauvignon (Paso Rob...,https://www.winemag.com/buying-guide/daou-2016...,Cabernet Sauvignon,2016,Daou
3,136,14.5,Red,US,"This wine is pretty, floral and compact in red...",NaN,102.0,$100+,95,91-100,California,Napa,Napa Valley,Alpha Omega 2015 Cabernet Sauvignon (Napa Valley),https://www.winemag.com/buying-guide/alpha-ome...,Cabernet Sauvignon,2015,Alpha Omega
4,161,14.7,Red,US,The producer's new Nomad collection includes b...,Nomad Beckstoffer Missouri Hopper Vineyard,175.0,$100+,95,91-100,California,Napa,Napa Valley,Stewart 2016 Nomad Beckstoffer Missouri Hopper...,https://www.winemag.com/buying-guide/stewart-2...,Cabernet Sauvignon,2016,Stewart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18051,237904,14.6,Red,US,Hard to explain why the winery is only now rel...,NaN,28.0,$25-$50,82,85-90,California,Central Coast,Santa Barbara County,Tin House 2003 Cabernet Sauvignon (Santa Barba...,https://www.winemag.com/buying-guide/tin-house...,Cabernet Sauvignon,2003,Tin House
18052,237909,13.0,Red,France,"A raw-edged wine, with dusty tannins and black...",Mouton Cadet,9.0,$0-$25,82,85-90,Bordeaux,Bordeaux,NaN,Baron Philippe de Rothschild 2005 Mouton Cadet...,https://www.winemag.com/buying-guide/baron-phi...,Bordeaux-style Red Blend,2005,Baron Philippe de Rothschild
18053,237910,13.8,Red,US,The blazing vintage took its toll on this high...,NaN,42.0,$25-$50,82,85-90,California,Sierra Foothills,Sierra Foothills,Peters Family 2004 Cabernet Sauvignon (Sierra ...,https://www.winemag.com/buying-guide/peters-fa...,Cabernet Sauvignon,2004,Peters Family
18054,237911,14.8,Red,US,"A brittle, peppery and ultimately disappointin...",Conn Creek Estate Vineyard,45.0,$25-$50,82,85-90,California,Napa,Rutherford,Conn Creek 2004 Conn Creek Estate Vineyard Cab...,https://www.winemag.com/buying-guide/conn-cree...,Cabernet Sauvignon,2004,Conn Creek


In [7]:
df = df.drop(['ID', 'alcohol', 'category', 'price_bin', 'description', 'designation', 'rating', 'title', 'url', 'varietal', 'vintage','winery'], axis=1)
df.head()


,country,price,rating_bin,region,subregion,subsubregion
0,US,185.0,91-100,California,Napa,St. Helena
1,US,35.0,91-100,California,Central Coast,Paso Robles
2,US,56.0,91-100,California,Central Coast,Paso Robles
3,US,102.0,91-100,California,Napa,Napa Valley
4,US,175.0,91-100,California,Napa,Napa Valley


In [8]:
df = df.dropna(subset=['subsubregion'])

In [9]:
X_df = pd.DataFrame(df)
X_df['subregion'] = X_df['subregion'].astype(str)

In [10]:
X = X_df.drop("rating_bin", axis=1)
y = df["rating_bin"]

In [11]:
X= X.apply(LabelEncoder().fit_transform)
X

,country,price,region,subregion,subsubregion
0,0,140,0,8,126
1,0,39,0,1,80
2,0,62,0,1,80
3,0,105,0,8,67
4,0,137,0,8,67
...,...,...,...,...,...
18049,0,5,0,1,80
18051,0,31,0,1,101
18053,0,46,0,13,110
18054,0,51,0,8,93


In [12]:
X = X.values.astype("float32")
print(X)
# y = y.values.astype("float32")

[[  0. 140.   0.   8. 126.]
 [  0.  39.   0.   1.  80.]
 [  0.  62.   0.   1.  80.]
 ...
 [  0.  46.   0.  13. 110.]
 [  0.  51.   0.   8.  93.]
 [  0.  16.   0.   1.  80.]]


In [13]:
le = LabelEncoder()
le.fit(y)
print(y)
encoded_y = le.transform(y)
encoded_y
le.inverse_transform(np.arange(2))

0        91-100
1        91-100
2        91-100
3        91-100
4        91-100
          ...  
18049     85-90
18051     85-90
18053     85-90
18054     85-90
18055     85-90
Name: rating_bin, Length: 13804, dtype: object


array(['85-90', '91-100'], dtype=object)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=42)

In [15]:
le2 = LabelEncoder()
le2.fit(y_train)
encoded_y_train = le2.transform(y_train)
encoded_y_test = le2.transform(y_test)
encoded_y_test
# len(encoded_y_train)

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [16]:
X_train, x_val, y_train, y_val = train_test_split(X_train, encoded_y_train, random_state=42)

In [17]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
print(len(y_train_categorical))
len(X_train)

7764


7764

In [18]:
scaler_model = StandardScaler()

In [19]:
X_scaler = scaler_model.fit(X_train)

In [20]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
input_dims = X_train.shape[1]
input_dims

5

In [22]:
model = Sequential()
model.add(Dense(8, activation = 'tanh', input_dim=(input_dims)))
# model.add(Dense(6, activation = 'relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 48        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit(X_train_scaled, y_train_categorical, epochs=300, batch_size=16,  verbose=1, validation_split=0.2)

Train on 6211 samples, validate on 1553 samples
Epoch 1/300
6211/6211 [==============================] - 0s 80us/step - loss: 0.5006 - accuracy: 0.7475 - val_loss: 0.4987 - val_accuracy: 0.7428
Epoch 2/300
6211/6211 [==============================] - 0s 57us/step - loss: 0.5003 - accuracy: 0.7467 - val_loss: 0.4988 - val_accuracy: 0.7463
Epoch 3/300
6211/6211 [==============================] - 0s 56us/step - loss: 0.5005 - accuracy: 0.7476 - val_loss: 0.4986 - val_accuracy: 0.7428
Epoch 4/300
6211/6211 [==============================] - 0s 54us/step - loss: 0.5003 - accuracy: 0.7464 - val_loss: 0.4988 - val_accuracy: 0.7424
Epoch 5/300
6211/6211 [==============================] - 0s 57us/step - loss: 0.5004 - accuracy: 0.7470 - val_loss: 0.4983 - val_accuracy: 0.7453
Epoch 6/300
6211/6211 [==============================] - 0s 65us/step - loss: 0.5002 - accuracy: 0.7463 - val_loss: 0.4986 - val_accuracy: 0.7450
Epoch 7/300
6211/6211 [==============================] - 0s 59us/step - loss

6211/6211 [==============================] - 0s 55us/step - loss: 0.4984 - accuracy: 0.7446 - val_loss: 0.4982 - val_accuracy: 0.7444
Epoch 57/300
6211/6211 [==============================] - 0s 56us/step - loss: 0.4981 - accuracy: 0.7480 - val_loss: 0.4986 - val_accuracy: 0.7382
Epoch 58/300
6211/6211 [==============================] - 0s 53us/step - loss: 0.4983 - accuracy: 0.7475 - val_loss: 0.4982 - val_accuracy: 0.7434
Epoch 59/300
6211/6211 [==============================] - 0s 54us/step - loss: 0.4981 - accuracy: 0.7472 - val_loss: 0.4987 - val_accuracy: 0.7376
Epoch 60/300
6211/6211 [==============================] - 0s 57us/step - loss: 0.4983 - accuracy: 0.7458 - val_loss: 0.4982 - val_accuracy: 0.7440
Epoch 61/300
6211/6211 [==============================] - 0s 54us/step - loss: 0.4980 - accuracy: 0.7479 - val_loss: 0.4985 - val_accuracy: 0.7386
Epoch 62/300
6211/6211 [==============================] - 0s 65us/step - loss: 0.4981 - accuracy: 0.7457 - val_loss: 0.4980 - val_a

Epoch 112/300
6211/6211 [==============================] - 0s 61us/step - loss: 0.4966 - accuracy: 0.7468 - val_loss: 0.4986 - val_accuracy: 0.7373
Epoch 113/300
6211/6211 [==============================] - 0s 59us/step - loss: 0.4968 - accuracy: 0.7472 - val_loss: 0.4979 - val_accuracy: 0.7405
Epoch 114/300
6211/6211 [==============================] - 0s 59us/step - loss: 0.4968 - accuracy: 0.7476 - val_loss: 0.4977 - val_accuracy: 0.7418
Epoch 115/300
6211/6211 [==============================] - 0s 58us/step - loss: 0.4965 - accuracy: 0.7459 - val_loss: 0.4977 - val_accuracy: 0.7460
Epoch 116/300
6211/6211 [==============================] - 0s 52us/step - loss: 0.4967 - accuracy: 0.7488 - val_loss: 0.4981 - val_accuracy: 0.7489
Epoch 117/300
6211/6211 [==============================] - 0s 55us/step - loss: 0.4965 - accuracy: 0.7487 - val_loss: 0.4977 - val_accuracy: 0.7498
Epoch 118/300
6211/6211 [==============================] - 0s 62us/step - loss: 0.4967 - accuracy: 0.7488 - val_

6211/6211 [==============================] - 0s 71us/step - loss: 0.4957 - accuracy: 0.7492 - val_loss: 0.4976 - val_accuracy: 0.7415
Epoch 168/300
6211/6211 [==============================] - 0s 68us/step - loss: 0.4958 - accuracy: 0.7482 - val_loss: 0.4979 - val_accuracy: 0.7428
Epoch 169/300
6211/6211 [==============================] - 0s 67us/step - loss: 0.4958 - accuracy: 0.7477 - val_loss: 0.4978 - val_accuracy: 0.7411
Epoch 170/300
6211/6211 [==============================] - 0s 65us/step - loss: 0.4956 - accuracy: 0.7480 - val_loss: 0.4977 - val_accuracy: 0.7434
Epoch 171/300
6211/6211 [==============================] - 0s 65us/step - loss: 0.4958 - accuracy: 0.7490 - val_loss: 0.4974 - val_accuracy: 0.7428
Epoch 172/300
6211/6211 [==============================] - 0s 77us/step - loss: 0.4956 - accuracy: 0.7469 - val_loss: 0.4980 - val_accuracy: 0.7415
Epoch 173/300
6211/6211 [==============================] - 0s 75us/step - loss: 0.4958 - accuracy: 0.7503 - val_loss: 0.4976 -

6211/6211 [==============================] - 0s 72us/step - loss: 0.4951 - accuracy: 0.7479 - val_loss: 0.4977 - val_accuracy: 0.7434
Epoch 223/300
6211/6211 [==============================] - 0s 71us/step - loss: 0.4951 - accuracy: 0.7473 - val_loss: 0.4978 - val_accuracy: 0.7428
Epoch 224/300
6211/6211 [==============================] - 0s 74us/step - loss: 0.4950 - accuracy: 0.7496 - val_loss: 0.4972 - val_accuracy: 0.7418
Epoch 225/300
6211/6211 [==============================] - 0s 69us/step - loss: 0.4953 - accuracy: 0.7476 - val_loss: 0.4972 - val_accuracy: 0.7444
Epoch 226/300
6211/6211 [==============================] - 0s 70us/step - loss: 0.4951 - accuracy: 0.7487 - val_loss: 0.4985 - val_accuracy: 0.7360
Epoch 227/300
6211/6211 [==============================] - 0s 74us/step - loss: 0.4954 - accuracy: 0.7485 - val_loss: 0.4974 - val_accuracy: 0.7440
Epoch 228/300
6211/6211 [==============================] - 0s 75us/step - loss: 0.4951 - accuracy: 0.7496 - val_loss: 0.4978 -

6211/6211 [==============================] - 0s 78us/step - loss: 0.4951 - accuracy: 0.7479 - val_loss: 0.4980 - val_accuracy: 0.7418
Epoch 278/300
6211/6211 [==============================] - 0s 75us/step - loss: 0.4945 - accuracy: 0.7486 - val_loss: 0.4979 - val_accuracy: 0.7405
Epoch 279/300
6211/6211 [==============================] - 1s 83us/step - loss: 0.4948 - accuracy: 0.7479 - val_loss: 0.4980 - val_accuracy: 0.7405
Epoch 280/300
6211/6211 [==============================] - 1s 110us/step - loss: 0.4946 - accuracy: 0.7486 - val_loss: 0.4981 - val_accuracy: 0.7418
Epoch 281/300
6211/6211 [==============================] - 1s 135us/step - loss: 0.4948 - accuracy: 0.7476 - val_loss: 0.4980 - val_accuracy: 0.7421
Epoch 282/300
6211/6211 [==============================] - 1s 87us/step - loss: 0.4946 - accuracy: 0.7481 - val_loss: 0.4981 - val_accuracy: 0.7431
Epoch 283/300
6211/6211 [==============================] - 1s 86us/step - loss: 0.4945 - accuracy: 0.7487 - val_loss: 0.4979

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = le.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {le.inverse_transform(list(y_test[:5]))}")
# print(f"X Values: {list(X_test[:5])}")


In [ ]:
# model.save('modelSS.h5')

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


In [ ]:
plot_history(history)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    


In [ ]:
## multiclass or binary report
## If binary (sigmoid output), set binary parameter to True
def full_multiclass_report(model,
                           x,
                           y_true,
                           classes,
                           batch_size=32,
                           binary=True):

    # 1. Transform one-hot encoded y_true into their class number
#     if not binary:
#         y_true = np.argmax(y_true,axis=1)
    
    # 2. Predict classes and stores in y_pred
    y_pred = model.predict_classes(x, batch_size=batch_size)
    
    # 3. Print accuracy score
    print("Accuracy : "+ str(accuracy_score(y_true,y_pred)))
    
    print("")
    
    # 4. Print classification report
    print("Classification Report")
    print(classification_report(y_true,y_pred,digits=5))    
    
    # 5. Plot confusion matrix
    cnf_matrix = confusion_matrix(y_true,y_pred)
    print(cnf_matrix)
    plot_confusion_matrix(cnf_matrix,classes=classes)

In [ ]:
print(x_val)
y_val

In [ ]:
full_multiclass_report(model,
                       x_val,
                       y_val,
                       le.inverse_transform(np.arange(2)))

In [ ]:
print(X_test_scaled)
y_test

In [ ]:
full_multiclass_report(model,
                       X_test_scaled,
                       y_test,
                       le.inverse_transform(np.arange(2)))

In [ ]:
# GridSearch
def create_model(dense_layers=[8],
                 activation='relu',
                 optimizer='adam'):
    model = Sequential()

    for index, lsize in enumerate(dense_layers):
        # Input Layer - includes the input_shape
        if index == 0:
            model.add(Dense(lsize,
                            activation=activation,
                            input_shape=(5,)))
        else:
            model.add(Dense(lsize,
                            activation=activation))
            
    model.add(Dense(2,activation='sigmoid'))
    model.compile(optimizer = optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model,
                        epochs=100, 
                        batch_size=16,
                        verbose=0)

param_grid = {'dense_layers': [[8],[10],[12]],
              'activation':['relu','tanh'],
              'optimizer':('rmsprop','adam'),
              'epochs':[100,300],
              'batch_size':[10,16]}

grid = GridSearchCV(model,
                    param_grid=param_grid,
                    return_train_score=True,
                    scoring=['precision_macro','recall_macro','f1_macro'],
                    refit='precision_macro')

grid_results = grid.fit(X_train_scaled,y_train)

print('Parameters of the best model: ')
print(grid_results.best_params_)